In [11]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from braindecode.datasets import TUHAbnormal
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows, create_windows_from_events, scale as multiply)

mne.set_log_level('ERROR')  # avoid messages everytime a window is extracted

TUH_PATH = '/home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0'
N_JOBS = 8  # specify the number of jobs for loading and windowing
N_SAMPLES = 10

tuh = TUHAbnormal(
    path=TUH_PATH,
    recording_ids=list(range(N_SAMPLES)),
    target_name=('pathological'),#'report'),
    preload=False,
    add_physician_reports=True,
    n_jobs=N_JOBS,  # Mock dataset can't
    # be loaded in parallel
)

print("length of dataset : ", len(tuh))

#show last example 
x, y = tuh[-1]
print('x:', x)
print('y:', y)



Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/041/00004196/s003_2009_09_03/00004196_s003_t000.edf...
EDF file detected
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/058/00005864/s001_2009_09_03/00005864_s001_t000.edf...
EDF file detected
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/009/00000929/s003_2009_09_04/00000929_s003_t002.edf...
EDF file detected
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/058/00005851/s001_2009_09_04/00005851_s001_t001.edf...
EDF file detected
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/059/00005909/s002_2009_09_09/00005909_s002_t000.edf...
EDF file detected
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/0

/home/jovyan/test_env/lib/python3.10/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/home/jovyan/test_env/lib/python3.10/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/home/jovyan/test_env/lib/python3.10/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/home/jovyan/test_env/lib/python3.10/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target

In [12]:
print(tuh.description)

                                                path  year  month  day  \
0  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9    3   
1  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9    3   
2  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9    4   
3  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9    4   
4  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9    9   
5  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9   10   
6  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9   10   
7  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9   11   
8  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9   15   
9  /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2....  2009      9   15   

   subject  session  segment  age gender  \
0     4196        3        0   53      F   
1     5864        1        0   30      M   
2      929        3        2   39      F   
3     585

In [13]:
# create windows

window_size_samples = 1000
window_stride_samples = 1000
tuh_windows = create_fixed_length_windows(
    tuh,
    window_size_samples=window_size_samples,
    window_stride_samples=window_stride_samples,
    drop_last_window=False,
    n_jobs=N_JOBS,

)


Loading data for 351 events and 1000 original time points ...
Loading data for 297 events and 1000 original time points ...
Loading data for 756 events and 1000 original time points ...
Loading data for 287 events and 1000 original time points ...
Loading data for 366 events and 1000 original time points ...
Loading data for 753 events and 1000 original time points ...
Loading data for 384 events and 1000 original time points ...
Loading data for 312 events and 1000 original time points ...
0 bad epochs dropped
Loading data for 322 events and 1000 original time points ...
0 bad epochs dropped
Loading data for 301 events and 1000 original time points ...
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped
0 bad epochs dropped


In [14]:
len(tuh_windows)

4129

In [15]:
splitted = tuh_windows.split("train")
train_set = splitted['True']
valid_set = splitted['False']

In [16]:
splitted

{'False': <braindecode.datasets.base.BaseConcatDataset at 0x7fcdefdaf640>,
 'True': <braindecode.datasets.base.BaseConcatDataset at 0x7fcdefdac610>}

In [17]:
for x,y,z in valid_set:
    print(y)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [18]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 2
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    model.cuda()

from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier
# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001

batch_size = 64
n_epochs = 50

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=None, epochs=n_epochs)

/home/jovyan/test_env/lib/python3.10/site-packages/braindecode/util.py:51: UserWarning: torch.backends.cudnn.benchmark was set to True which may results in lack of reproducibility. In some cases to ensure reproducibility you may need to set torch.backends.cudnn.benchmark to False.
  warn(


  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  ----------------  ------------  ------  -------
      1            0.6344        0.8274            0.5768        1.4776  0.0006  25.7319
      2            0.7305        0.5715            0.7004        1.0946  0.0006  17.2383
      3            0.7452        0.4900            0.7814        0.8651  0.0006  18.1833
      4            0.7709        0.4318            0.8565        0.9921  0.0006  19.5855
      5            0.7643        0.3777            0.5764        1.0709  0.0006  21.7872
      6            0.7709        0.3607            0.8272        1.3236  0.0006  18.6834
      7            0.7709        0.3450            0.8493        1.0487  0.0006  16.6331
      8            0.7805        0.3671            0.6588        1.2468  0.0006  17.1879
      9            0.7819        0.3613            0.6357        1.6572  0.0006  17.7830
     10            0.